In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def build_model(num_classes):
    '''
    Создание и компиляция модели сверточной нейронной сети (CNN) для классификации изображений.

    Args:
        num_classes (int): Количество классов в задаче классификации.

    Returns:
        Sequential: скомпилированная модель Keras.
    '''

    model = Sequential([
        # Входной слой, принимает изображения размером 28x28 пикселей с одним каналом (черно-белые).
        Input(shape=(28, 28, 1), name='input_layer'),

        # Первый сверточный слой с 32 фильтрами размером 3x3 и функцией активации ReLU.
        # Этот слой извлекает основные признаки из изображения.
        Conv2D(32, (3, 3), activation='relu', name='conv2d_layer1'),

        # Слой максимального пулинга с размером пула 2x2, уменьшает размерность пространственных данных.
        # Это помогает уменьшить переобучение, сохраняя при этом важные признаки.
        MaxPooling2D((2, 2), name='max_pooling2d_layer1'),

        # Слой исключения с коэффициентом 0.25, помогает предотвратить переобучение.
        Dropout(0.25, name='dropout_layer1'),

        # Преобразование многомерных признаков из предыдущих сверточных и пулинговых слоев в одномерный вектор.
        Flatten(name='flatten_layer'),

        # Полносвязный слой с 256 нейронами и функцией активации ReLU.
        Dense(300, activation='relu', name='dense_layer1'),

        # Второй слой исключения с коэффициентом 0.25.
        Dropout(0.25, name='dropout_layer2'),

        # Выходной полносвязный слой, использует функцию активации Softmax для классификации на num_classes классов.
        Dense(num_classes, activation='softmax', name='output_layer')
    ])

    # Компиляция модели с использованием оптимизатора Adam.
    # 'categorical_crossentropy' используется для многоклассовой классификации,
    # 'accuracy' используется для оценки процента правильно классифицированных изображений.
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model



# Загрузка данных и меток из файла
data = np.load('fonts_dataset.npz')
X_train, X_test, y_train, y_test = data['X_train'], data['X_test'], data['y_train'], data['y_test']
labels = data['labels']

# Определение количества классов (шрифтов)
num_classes = len(set(labels))
model = build_model(num_classes)

# Печать сводки модели
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_layer1 (Conv2D)          │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_layer1            │ (None, 13, 13, 32)     │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_layer1 (Dropout)        │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_layer (Flatten)         │ (None, 5408)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_layer1 (Dense)            │ (None, 300)            │     1,622,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_layer2 (Dropout)        │ (None, 300)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 15)             │         4,515 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,627,535 (6.21 MB)

 Trainable params: 1,627,535 (6.21 MB)

 Non-trainable params: 0 (0.00 B)

In [2]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from sklearn.metrics import classification_report, confusion_matrix


def train_model(model, X_train, y_train, X_test, y_test, epochs=12, batch_size=220):
    '''Обучение модели и вывод основных метрик классификации'''
    
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                        validation_data=(X_test, y_test), verbose=2)
    
    # Логгирование результатов в DataFrame
    history_df = pd.DataFrame(history.history)
    history_df.to_csv('training_log.csv', index=False)

    # Оценка модели
    loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test loss: {loss:.3f}, Test accuracy: {accuracy:.3f}")
    
    # Предсказание классов для тестовых данных
    y_pred = np.argmax(model.predict(X_test), axis=1)

    # Вывод отчета о классификации и матрицы ошибок
    print("Classification Report:")
    print(classification_report(y_test_encoded, y_pred))

    print("Confusion Matrix:")
    print(confusion_matrix(y_test_encoded, y_pred))
    
    model.save("font_recognition_model.keras")

    return model


# Подготовка данных
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train.ravel())
y_test_encoded = label_encoder.transform(y_test.ravel())
y_train_onehot = to_categorical(y_train_encoded, num_classes)
y_test_onehot = to_categorical(y_test_encoded, num_classes)

# Сохранение данных кодировщика меток
with open('label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)
    
# Нормализация данных
X_train_normalized = X_train.astype('float32') / 255.0
X_test_normalized = X_test.astype('float32') / 255.0
X_train_normalized = np.expand_dims(X_train_normalized, axis=-1)
X_test_normalized = np.expand_dims(X_test_normalized, axis=-1)

# Обучение модели
model = train_model(model, X_train_normalized, y_train_onehot, X_test_normalized, y_test_onehot)

Epoch 1/12
253/253 - 6s - 24ms/step - accuracy: 0.7856 - loss: 0.6974 - val_accuracy: 0.9253 - val_loss: 0.2595
Epoch 2/12
253/253 - 5s - 22ms/step - accuracy: 0.9266 - loss: 0.2371 - val_accuracy: 0.9473 - val_loss: 0.1728
Epoch 3/12
253/253 - 5s - 21ms/step - accuracy: 0.9485 - loss: 0.1658 - val_accuracy: 0.9625 - val_loss: 0.1241
Epoch 4/12
253/253 - 5s - 21ms/step - accuracy: 0.9582 - loss: 0.1313 - val_accuracy: 0.9702 - val_loss: 0.1006
Epoch 5/12
253/253 - 5s - 21ms/step - accuracy: 0.9649 - loss: 0.1090 - val_accuracy: 0.9708 - val_loss: 0.0902
Epoch 6/12
253/253 - 5s - 21ms/step - accuracy: 0.9690 - loss: 0.0947 - val_accuracy: 0.9750 - val_loss: 0.0785
Epoch 7/12
253/253 - 6s - 22ms/step - accuracy: 0.9717 - loss: 0.0835 - val_accuracy: 0.9768 - val_loss: 0.0733
Epoch 8/12
253/253 - 5s - 21ms/step - accuracy: 0.9738 - loss: 0.0770 - val_accuracy: 0.9776 - val_loss: 0.0677
Epoch 9/12
253/253 - 5s - 21ms/step - accuracy: 0.9759 - loss: 0.0703 - val_accuracy: 0.9781 - val_loss: